In [3]:
import numpy as np
import pandas as pd
import sys
from scipy.special import comb

NBA_YEARS = range(1950, 2021)

In [7]:
def generate_collapse_scores_by_year(league, year, flat=True):
    df = pd.read_csv("data/" + league + "/" + str(year) + ".csv")
    
    season = {}
    
    for index, row in df.iterrows():
        home_team = row["home_team"]
        away_team = row["away_team"]
        home_win = row["home_team_score"] > row["away_team_score"]
               
        season[home_team] = np.append(season.get(home_team, np.array([])), home_win)
        season[away_team] = np.append(season.get(away_team, np.array([])), not home_win)
        
    # remove playoffs by finding minimum games played and only retaining those    
    
    regular_season_length = sys.maxsize
    
    for team in season:
        num_team_games = len(season[team])
        if num_team_games < regular_season_length:
            regular_season_length = num_team_games
            
    for team in season:
        season[team] = season[team][:regular_season_length]
        
    season_end_length = 10 if flat else round(regular_season_length / 2)
        
    # calculate collapse scores    
        
    collapse_scores = {}
    
    for team in season:
        games = season[team]
        
        win_pct = np.sum(games) / len(games)
        season_end_wins = int(np.sum(games[-season_end_length:]))
        
        probability = 0
        
        for wins in range(0, season_end_wins + 1):
            temp = comb(season_end_length, wins) * win_pct ** wins * (1 - win_pct) ** (season_end_length - wins)
            probability += temp
            
        collapse_scores[team] = 1 / probability
        
    return collapse_scores

def generate_collapse_scores(league, years):
    collapse_scores = {}
    team_years = {}
    
    for year in years:
        year_collapse_scores = generate_collapse_scores_by_year(league, year)
        
        for team in year_collapse_scores:
            collapse_scores[team] = np.append(collapse_scores.get(team, np.array([])), year_collapse_scores[team])
            team_years[team] = np.append(team_years.get(team, np.array([])), year)
            
    return collapse_scores, team_years

In [8]:
generate_collapse_scores("nba", NBA_YEARS)

({'TRI-CITIES BLACKHAWKS': array([1.14161241, 1.12425107]),
  'DENVER NUGGETS': array([7.05042235, 1.68644852, 1.52971419, 1.03340435, 2.12288125,
         1.02869198, 4.18911933, 1.35770084, 2.12944174, 1.89056612,
         4.6608844 , 1.11534569, 1.10481977, 3.43858484, 1.27357489,
         3.87535977, 6.20969569, 1.31356619, 1.23856498, 1.20754717,
         1.77116093, 4.3371128 , 1.1633639 , 2.28399653, 1.08401214,
         1.18007856, 1.07813902, 2.8239074 , 1.27357489, 1.04701295,
         3.43858484, 1.0024835 , 1.68644852, 1.10481977, 2.01573275,
         1.22479266, 1.03466076, 1.16531705, 1.8763276 , 2.12288125,
         2.65828914, 1.53217031, 1.12651082, 4.34627927, 4.93619273]),
  'CHICAGO STAGS': array([11.61462361]),
  'NEW YORK KNICKS': array([  1.70981031,   1.33535403,   1.12594304,   2.49247015,
           1.69640947,   1.08353118,   1.17652183,   1.20754717,
           1.52272491,   1.38493954,  13.09225265,   4.75646359,
           2.08312667,   4.60765003,   2.212